In [1]:
inputPath = "dbfs:/mnt/yelp-mount"
userDf = spark.read.json(inputPath+"/user.json")
reviewDf = spark.read.json(inputPath+"/review.json")

In [2]:
filteredBusinessDf = spark.read.json('/FileStore/tables/filtered_business.json')

In [3]:
filteredBusinessDf.createOrReplaceTempView("business")
userDf.createOrReplaceTempView("user")
reviewDf.createOrReplaceTempView("review")

In [4]:
reviewDf.head()

In [5]:
userDf.head()

In [6]:
display(filteredBusinessDf)

In [7]:
df = sqlContext.sql('select * from review inner join business on business.business_id = review.business_id inner join user on user.user_id = review.user_id')